## TRABAJO FINAL 

In [7]:
import numpy as np
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics, neighbors, tree, preprocessing
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import ListedColormap
from shutil import copyfile
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score 
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import math 
import category_encoders as ce
from sklearn import pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn import feature_selection

## TEMA 3: METRICAS RENDIMIENTO

In [5]:
def obtenerMatrizConfusion(clasificador, X, y):
    # Habra que ajustar labels al numero de clases que tengamos
    matrizConfusion = confusion_matrix(y,clasificador.predict(X), labels= [1,0])
    return matrizConfusion

def calidad_modelo(clf, X,y):
    y_pred = clf.predict(X)
    recall_medio_clf = roc_auc_score(y, y_pred)*100
    tpr_clf = recall_score(y,y_pred, pos_label=1)
    tnr_clf = recall_score(y,y_pred, pos_label=0)
    gm_clf = math.sqrt(tpr_clf*tnr_clf)*100
    fscore_clf = f1_score(y,y_pred)*100
    return recall_medio_clf,gm_clf, fscore_clf

def resumenMedidasRendimient(clasificador, X,y):
    return classification_report(y,clasificador.predict(X))

# labels debe ser un listado con los nombres de los clasificadores.
def muestra_ROC_courve(listaClfs, X, y, labels):
    # generamos una predicción sin calidad para la clase positiva (1): todas las predicciones con probabilidad 0
    ns_probs = np.zeros(len(y))
    # calculamos el área bajo la curva ROC (método roc_auc_score pasando las clases reales y las probabilidades de la clase positiva)
        # En tanto por 100
    ns_auc = roc_auc_score(y,ns_probs)*100
    # calculamos todos los pares de puntos (fpr, tpr) para dibujar la curva ROC (método roc_curve pasando las clases reales y las probabilidades de la clase positiva)
    ns_fpr, ns_tpr, _ = roc_curve(y,ns_probs)
    
    # Lista para almacenar los AUC bajo la curva ROC de los clasificadores
    listaAUCs = []
    # Para cada clasificador de la lista
    for i,clf in enumerate(listaClfs):
        # predecimos las probabilidades de predecir cada ejemplo en cada clase en base al método que implemente el clasificador
        if hasattr(clf, 'predict_proba'):
            # Si el clasificador implementa predict_proba nos quedamos con las predicciones de la clase positiva (1)
            model_probs = clf.predict_proba(X)[:,1]
        else:
            # Si el clasificador implementa decision_function
            model_probs = clf.decision_function(X)

        # calculamos todos los pares de puntos (fpr, tpr) para dibujar la curva ROC (método roc_curve pasando las clases reales y las probabilidades de la clase positiva)
        model_fpr, model_tpr, _ = roc_curve(y,model_probs)
        # calculamos el área bajo la curva ROC, y lo añadimos a la lista de rendimiento, en tanto por 100
        model_auc = roc_auc_score(y,model_probs)*100
        listaAUCs.append(model_auc)
        # Mostramos visualmente la curva ROC (plot mostrando fpr en el eje x y tpr en el eje y) poniendo como etiqueta el nombre del clasificador
        plt.figure()
        plt.title(labels[i])
        plt.plot(model_fpr,model_tpr)
        
    # mostramos la curva del modelo sin calidad
    plt.figure()
    plt.plot(ns_fpr,ns_tpr)
    # Etiquetamos los ejes
    plt.xlabel("fpr")
    plt.ylabel("tpr")
    # Creamos la leyenda de la figura
    plt.legend()
    # Ponemos título a la figura
    plt.title("Modelo sin calidad")
    # Mostramos la figura
    # <RELLENAR>
    # Devolvemos la lista con los AUC de los diferentes clasificadores
    return listaAUCs


# labels debe ser un listado con los nombres de los clasificadores.
def muestra_PR_courve(listaClfs, X, y, labels):
    # Calculamos lo necesario para mostrar la curva PR y sus valores de rendimiento
        # Predicción sin calidad para todos los ejemplos (proporción de ejemplos de la clase positiva, 1) 
    no_skill = np.zeros(len(y))
    precision,recall,_=precision_recall_curve(y,no_skill)
    plt.figure()
    plt.title("Sin calidad")
    plt.plot(precision,recall)
    plt.xlabel("precision")
    plt.xlabel("recall")
    # Para cada clasificador de la lista
    rendimientos = []
    for i,clf in enumerate(listaClfs):
        if hasattr(clf, 'predict_proba'):
            # Si el clasificador implementa predict_proba nos quedamos con las predicciones de la clase positiva (1)
            model_probs = clf.predict_proba(X)[:,1]
        else:
            # Si el clasificador implementa decision_function
            model_probs = clf.decision_function(X)
        precision, recall,_ = precision_recall_curve(y,model_probs)
        plt.figure()
        plt.title(labels[i])
        plt.plot(precision,recall, label = "PR")
        plt.xlabel("precision")
        plt.xlabel("recall")
        plt.legend()
        rendimientos.append(auc(recall,precision)*100)
    return rendimientos


# Función que muestra el compromiso entre precision y recall para un clasificador (clf) y unos datos de entrada (X) y salida (y)
def muestra_balance_PR(clf, X, y):
    # Obtenemos las probabilidades de la clase positiva
    model_probs = clf.predict_proba(X)[:,1]
    
    # Obtenemos los umbrales que se generan y que caracterizan la curva PR (usar método precision_recall_curve)
    model_precision, model_recall, umbrales = precision_recall_curve(y,model_probs)
    # Añadimos 1 como umbral para que la gráfica vaya hasta dicho punto (el par PR ya está calculado)
    umbrales = np.insert(umbrales, umbrales.size, 1., axis=0)
    # Mostramos la figura: la precisión en rojo y el recall en azul
    plt.figure()
    plt.title("Balance precision-recall")
    plt.plot(model_precision,model_recall)
    plt.xlabel("precision")
    plt.xlabel("recall")

    
def clasificacion_umbral(probs_clase_positiva, umbral=0.5):
    salida = []
    for i in probs_clase_positiva:
        if i > umbral:
            salida.append(1)
        else:
            salida.append(0)
    return salida

def calcular_mejor_umbral_fscore(model, X_train, y_train, X_val, y_val):    
    # predecimos las probabilidades de predecir cada ejemplo de train y de validación en la clase positiva
    model_probs = model.predict_proba(X_train)[:,1]
    model_probs_val =  model.predict_proba(X_val)[:,1]
    
    # calculamos todos los pares de puntos (recall, precision) y sus umbrales (método precision_recall_curve)
    model_precision, model_recall, umbrales = precision_recall_curve(y_train,model_probs)
    # Evaluamos el Fscore (método f1_score) de cada umbral 
    rendimiento_umbrales = []
    indiceMejorUmbral = 0
    max_fscore = 0
    for i,umbral in enumerate(umbrales):
        fscore = evaluar_mejor_umbral_fscore(model,umbral,X_val,y_val)
        rendimiento_umbrales.append(fscore)
        if (fscore > max_fscore):
            indiceMejorUmbral = i
            mejorUmbral = umbral
            max_fscore = fscore 
    # Se consigue el índice del umbral que da mayor rendimiento
#     indiceMejorUmbral = <RELLENAR>
    # Se consigue el valor del mejor umbral
#     mejorUmbral = <RELLENAR>
    print('Mejor umbral={:.3f}, Fscore en validacion={:.2f}'.format(mejorUmbral, rendimiento_umbrales[indiceMejorUmbral]))
    # Se devuelve el mejor umbral
    return mejorUmbral

def evaluar_mejor_umbral_fscore(model, umbral, X, y):
    # Evaluamos la calidad del mejor umbral con los ejemplos
    prediccion = model.predict_proba(X)[:,1]
    model_probs = clasificacion_umbral(prediccion, umbral)
    fscore = f1_score(y,model_probs)
    return fscore*100

def calcular_mejor_umbral_AUC_ROC(model, X_train, y_train, X_val, y_val):    
    # predecimos las probabilidades de predecir cada ejemplo de train y de validación en la clase positiva
    model_probs = model.predict_proba(X_train)[:,1]
    model_probs_val =  model.predict_proba(X_val)[:,1]
    
    # calculamos todos los pares de puntos (recall, precision) y sus umbrales (método precision_recall_curve)
    model_precision, model_recall, umbrales = roc_curve(y_train,model_probs)

    # Evaluamos el Fscore (método f1_score) de cada umbral 
    rendimiento_umbrales = []
    indiceMejorUmbral = 0
    max_auc_roc = 0
    
    for i,umbral in enumerate(umbrales):
        auc_roc = evaluar_mejor_umbral_AUC_ROC(model,umbral,X_val,y_val)
        rendimiento_umbrales.append(auc_roc)
        if (auc_roc > max_auc_roc):
            indiceMejorUmbral = i
            mejorUmbral = umbral
            max_auc_roc = auc_roc 
    # Se consigue el índice del umbral que da mayor rendimiento
#     indiceMejorUmbral = <RELLENAR>
    # Se consigue el valor del mejor umbral
#     mejorUmbral = <RELLENAR>
    print('Mejor umbral={:.3f}, AUC en validacion={:.2f}'.format(mejorUmbral, rendimiento_umbrales[indiceMejorUmbral]))
    # Se devuelve el mejor umbral
    return mejorUmbral

def evaluar_mejor_umbral_AUC_ROC(model, umbral, X, y):
    prediccion = model.predict_proba(X)[:,1]
    model_probs = clasificacion_umbral(prediccion, umbral)
    auc_roc = roc_auc_score(y,model_probs)
    return auc_roc*100


def tunearClasificador(clf, dic, metrica_rendimiento, X,y):
    np.random.seed(12)
    validacionCruzada = model_selection.StratifiedKFold(n_splits=5)
    grid = GridSearchCV(clf, param_grid= dic,scoring=metrica_rendimiento, cv = validacionCruzada,return_train_score=True)
    grid.fit(X,y)
    resultadosMostrar = zip(grid.cv_results_['params'],grid.cv_results_['mean_test_score'],grid.cv_results_['mean_train_score'])
    for params, mean_test_score, mean_train_score in resultadosMostrar:
        print("%0.3f (Train: %0.3f) for %r" % (mean_test_score, mean_train_score, params))
        print()

    return grid.best_estimator_, grid.best_score_, grid.best_params_

# Esto es un ejmplo: 
score_PR = make_scorer(average_precision_score,needs_threshold=True)
LR = LogisticRegression()
grid_LR = {'C': [0.1,1,3,6,10]}
bestLR_PR, best_val, best_hipPar = tunearClasificador(LR,grid_LR,score_PR,X,Y)

## TEMA 4: IMPUTACION DE MISSING VALUES

In [ ]:
preprocesamiento = ColumnTransformer(transformers=(("media", SimpleImputer(strategy='mean') , nombres_variables_numericas), 
                                                    ("moda", SimpleImputer(strategy='most_frequent'), nombres_variables_categoricas)))

autos_sin_missing = preprocesamiento.fit_transform(autos_sin_missing)

In [ ]:
# Realizamos la transformación utilizando la codificación ordinal
ordinalEncoder = ce.OrdinalEncoder(cols = None)
autos_ord = ordinalEncoder.fit_transform(autos_ord)

In [ ]:
# Realizamos la transformación utilizando la codificación del conteo
autos_count = autos_sin_missing.copy()

countEncoder = ce.CountEncoder(cols = None)
autos_count = countEncoder.fit_transform(autos_count)

In [ ]:
# Realizamos la transformación utilizando la codificación One Hot
autos_oh = autos_sin_missing.copy()

oneHotEncoder = ce.OneHotEncoder(cols = None)
autos_oh = oneHotEncoder.fit_transform(autos_oh)

In [ ]:
# Realizamos la transformación utilizando la codificación binaria
autos_binary = autos_sin_missing.copy()

ordinalEncoder = ce.OrdinalEncoder(cols = nombres_variables_categoricas)
binaryEncoder = ce.BinaryEncoder(cols = nombres_variables_categoricas)

autos_binary = ordinalEncoder.fit_transform(autos_binary)
autos_binary = binaryEncoder.fit_transform(autos_binary)

In [ ]:
# Realizamos la transformación utilizando la codificación basada en la salida 
autos_te = autos_sin_missing.copy()
salidas_te = autos_output.copy()

targetEncoder = ce.TargetEncoder(smoothing=0.0000001)
salidas_te = pd.DataFrame(autos_output, columns = ['price'])
salidas_te = salidas_te.set_index(autos_te.index)

autos_te = targetEncoder.fit_transform(autos_te, salidas_te)

In [ ]:
# Ejemplo de pipeline  + Grid search: 
pipe = pipeline.Pipeline([('codificacion ordinal ', ce.OrdinalEncoder(cols = nombres_variables_categoricas)), ('estandarizacion', MinMaxScaler()), ('knn', KNeighborsRegressor()) ])
hiperParameters =  {
    "modelo__n_neighbors": [3,5,7,9],
    "modelo__weights": ['uniform','distance'],
    "modelo__p":[1,2,1.5,3]
}
ss = model_selection.ShuffleSplit(n_splits = 1, test_size= 0.2)
gridSearch_pipeline = model_selection.GridSearchCV(pipe, hiperParameters,cv=ss, return_train_score=True, scoring='neg_root_mean_squared_error')

In [ ]:
# Clasificar las variables categoricas segun el numero de valores que toman.

nombres_variables_numericas = nombres_variables_numericas
nombres_variables_categoricas_2_valores = []
nombres_variables_categoricas_entre3y6_valores = []
nombres_variables_categoricas_masDe6_valores = []
for column in nombres_variables_categoricas:
    valores = len(pd.unique(autos_sin_missing[column]))
    if (valores <=2 ):
        nombres_variables_categoricas_2_valores.append(column)
    if (valores >2 and valores<=6 ):
        nombres_variables_categoricas_entre3y6_valores.append(column)
    if(valores >6 ):
        nombres_variables_categoricas_masDe6_valores.append(column)

print(nombres_variables_numericas)
print(nombres_variables_categoricas_2_valores)
print(nombres_variables_categoricas_entre3y6_valores)
print(nombres_variables_categoricas_masDe6_valores)

# Creamos un imput_transformer para cada tipo de variable categorica.

categorical_2_imputer_transformer = pipeline.Pipeline([
    ("imputacion por la moda ", SimpleImputer(strategy='most_frequent')),
    ("Codificacion ordinal ", ce.OrdinalEncoder())]
)
categorical_entre3y6_imputer_transformer = pipeline.Pipeline([
    ("imputacion por la moda ", SimpleImputer(strategy='most_frequent')),
    ("Codificacion one hot encoding ", ce.OneHotEncoder())]
)
categorical_masDe6_imputer_transformer = pipeline.Pipeline([
    ("imputacion por la moda ", SimpleImputer(strategy='most_frequent')),
    ("Codificacion basada en saldia ", ce.TargetEncoder(smoothing=0.0000001))]
)

preprocesamiento = ColumnTransformer(
    transformers=[
        ("media", SimpleImputer(strategy='mean') , nombres_variables_numericas),
        ("Categoricas 2 valores",categorical_2_imputer_transformer,  nombres_variables_categoricas_2_valores),
        ("Categoricas 3-6 valores",categorical_entre3y6_imputer_transformer,  nombres_variables_categoricas_entre3y6_valores),
        ("Categoricas mas de 6", categorical_masDe6_imputer_transformer, nombres_variables_categoricas_masDe6_valores)
    ]
)

# Pasamos el preprocesamiento creado a una pipeline.
pipe = pipeline.Pipeline([('pre procesamiento ', preprocesamiento), ('minMaxScaler', MinMaxScaler()), ('knn', KNeighborsRegressor()) ])

## TEMA 5: ARBOLES DE DECISION 

In [8]:
# NO SABEMOS SI VAMOS A IMPLEMENTAR ARBOLES

## TEMA 6: FILTROS Y WRAPPERS (SELECCION DE VARIABLES)

SELECCION DE LAS K MEJORES VARIABLES ( UTILIZANDO CHI )

In [ ]:
# Listas para almacenar los resultados de accuracy en train y test
listaAccTrain = []
listaAccTest = []
for numVar in [10,20,30]:
# <RELLENAR>
    # Se llama al constructor que realiza la selección de las k mejores variables con los parámetros apropiados
    tecnicaSeleccion = feature_selection.SelectKBest(score_func=feature_selection.chi2, k = numVar)
    # Llamada a la función que aprende los parámetros de la selección de variables a partir de los datos de entrenamiento
    tecnicaSeleccion =  tecnicaSeleccion.fit(X_train,y_train.Type)
    # Llamada a la función que transforma los datos de entrenamiento: realiza la selección de variables
    X_train_seleccion = tecnicaSeleccion.transform(X_train)
    # Llamada a la función que transforma los datos de test: realiza la selección de variables
    X_test_seleccion =  tecnicaSeleccion.transform(X_test)
    # Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
    knn =  neighbors.KNeighborsClassifier()
    # Llamada a la función que realiza el aprendizaje del clasificador
    knn =  knn.fit(X_train_seleccion,y_train.Type)
    # Llamada a la función que realiza la predicción de los datos de entrenamiento
    prediccionTrain =  knn.predict(X_train_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
    accTrain =  metrics.accuracy_score(y_train,prediccionTrain)*100
    # Se añade el resultado a la lista de resultados de train
    listaAccTrain.append(accTrain)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en entrenamiento'.format(numVar, accTrain))
    # Llamada a la función que realiza la predicción de los datos de test
    prediccionTest =  knn.predict(X_test_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de test
    accTest =  metrics.accuracy_score(y_test,prediccionTest)*100
    # Se añade el resultado a la lista de resultados de test
    listaAccTest.append(accTest)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en test'.format(numVar, accTest))

In [ ]:
listaAccTrain = [] # lista para almancenar el rendimiento de las diferentes posibilidades en train
listaAccTest = [] # lista para almancenar el rendimiento de las diferentes posibilidades en test
for numVar in [10,20,30]:
# <RELLENAR>
    # Se llama al constructor que realiza la selección de las k mejores variables con los parámetros apropiados
    tecnicaSeleccion = feature_selection.SelectKBest(score_func=feature_selection.f_classif, k = numVar)
    # Llamada a la función que aprende los parámetros de la selección de variables a partir de los datos de entrenamiento
    tecnicaSeleccion =  tecnicaSeleccion.fit(X_train,y_train.Type)
    # Llamada a la función que transforma los datos de entrenamiento: realiza la selección de variables
    X_train_seleccion = tecnicaSeleccion.transform(X_train)
    # Llamada a la función que transforma los datos de test: realiza la selección de variables
    X_test_seleccion =  tecnicaSeleccion.transform(X_test)
    # Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
    knn =  neighbors.KNeighborsClassifier()
    # Llamada a la función que realiza el aprendizaje del clasificador
    knn =  knn.fit(X_train_seleccion,y_train.Type)
    # Llamada a la función que realiza la predicción de los datos de entrenamiento
    prediccionTrain =  knn.predict(X_train_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
    accTrain =  metrics.accuracy_score(y_train,prediccionTrain)*100
    # Se añade el resultado a la lista de resultados de train
    listaAccTrain.append(accTrain)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en entrenamiento'.format(numVar, accTrain))
    # Llamada a la función que realiza la predicción de los datos de test
    prediccionTest =  knn.predict(X_test_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de test
    accTest =  metrics.accuracy_score(y_test,prediccionTest)*100
    # Se añade el resultado a la lista de resultados de test
    listaAccTest.append(accTest)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en test'.format(numVar, accTest))

In [ ]:
# En vez de elegir numero de variables que queremos, ahora metodo del percentil 

listaAccTrain = [] # lista para almancenar el rendimiento de las diferentes posibilidades en train
listaAccTest = [] # lista para almancenar el rendimiento de las diferentes posibilidades en test
for numVar in [10,20,30]:
# <RELLENAR>
    # Se llama al constructor que realiza la selección de las k mejores variables con los parámetros apropiados
    tecnicaSeleccion = feature_selection.SelectPercentile(score_func=feature_selection.chi2, percentile = numVar)
    # Llamada a la función que aprende los parámetros de la selección de variables a partir de los datos de entrenamiento
    tecnicaSeleccion =  tecnicaSeleccion.fit(X_train,y_train.Type)
    # Llamada a la función que transforma los datos de entrenamiento: realiza la selección de variables
    X_train_seleccion = tecnicaSeleccion.transform(X_train)
    # Llamada a la función que transforma los datos de test: realiza la selección de variables
    X_test_seleccion =  tecnicaSeleccion.transform(X_test)
    # Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
    knn =  neighbors.KNeighborsClassifier()
    # Llamada a la función que realiza el aprendizaje del clasificador
    knn =  knn.fit(X_train_seleccion,y_train.Type)
    # Llamada a la función que realiza la predicción de los datos de entrenamiento
    prediccionTrain =  knn.predict(X_train_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
    accTrain =  metrics.accuracy_score(y_train,prediccionTrain)*100
    # Se añade el resultado a la lista de resultados de train
    listaAccTrain.append(accTrain)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en entrenamiento'.format(numVar, accTrain))
    # Llamada a la función que realiza la predicción de los datos de test
    prediccionTest =  knn.predict(X_test_seleccion)
    # Llamada a la función que calcula el porcentaje de acierto para los datos de test
    accTest =  metrics.accuracy_score(y_test,prediccionTest)*100
    # Se añade el resultado a la lista de resultados de test
    listaAccTest.append(accTest)
    print('Seleccionando las {} mejores variables se obtiene un accuracy del {}% en test'.format(numVar, accTest))

SELECCION DE LAS K MEJORES VARIABLES ( UTILIZANDO ANOVA )

## TEMA 7: SELECCION DE INSTANCIAS Y MUESTREO 